<a href="https://colab.research.google.com/github/HarinduR/FeatherFind/blob/Keyword-Bird-Finder/FeatherFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

def parse_user_input(text):
    doc = nlp(text)
    dependencies = []
    for token in doc:
        dependencies.append((token.text, token.dep_, token.head.text))
    return dependencies

# Example usage
user_input = "A small bird with red wings in forests."
dependencies = parse_user_input(user_input)
print("Dependencies:", dependencies)

Dependencies: [('A', 'det', 'bird'), ('small', 'amod', 'bird'), ('bird', 'ROOT', 'bird'), ('with', 'prep', 'bird'), ('red', 'amod', 'wings'), ('wings', 'pobj', 'with'), ('in', 'prep', 'bird'), ('forests', 'pobj', 'in'), ('.', 'punct', 'bird')]


In [24]:
def extract_features(dependencies):
    features = {
        "size": None,
        "color": None,
        "habitat": None,
        "anatomy": []
    }

    for word, dep, head in dependencies:
        if dep == "amod" and head == "bird":
            features["size"] = word  # e.g., "small"
        elif dep == "amod" and head == "wings":
            features["color"] = word  # e.g., "red"
        elif dep == "pobj" and head == "in":
            features["habitat"] = word  # e.g., "forests"
        elif dep == "pobj" and head == "with":
            features["anatomy"].append(word)  # e.g., "wings"

    return features

# Example usage
features = extract_features(dependencies)
print("Extracted Features:", features)

Extracted Features: {'size': 'small', 'color': 'red', 'habitat': 'forests', 'anatomy': ['wings']}


In [25]:
def map_to_ontology(features):
    ontology_mapping = {
        "size": "bird:hasSize",
        "color": "bird:hasColor",
        "habitat": "bird:inhabits",
        "anatomy": "bird:hasAnatomicalFeature"
    }

    sparql_filters = []
    for key, value in features.items():
        if value:
            if key == "anatomy":
                for item in value:
                    sparql_filters.append(f'{ontology_mapping[key]} "{item}"')
            else:
                sparql_filters.append(f'{ontology_mapping[key]} "{value}"')

    return sparql_filters

# Example usage
sparql_filters = map_to_ontology(features)
print("SPARQL Filters:", sparql_filters)

SPARQL Filters: ['bird:hasSize "small"', 'bird:hasColor "red"', 'bird:inhabits "forests"', 'bird:hasAnatomicalFeature "wings"']


In [26]:
def generate_sparql_query(filters):
    query = """
    PREFIX bird: <http://example.org/bird-ontology#>
    SELECT ?bird WHERE {{
        ?bird a bird:Species ;
              {} .
    }}
    """.format(" ; ".join(filters))
    return query

# Example usage
sparql_query = generate_sparql_query(sparql_filters)
print("Generated SPARQL Query:\n", sparql_query)

Generated SPARQL Query:
 
    PREFIX bird: <http://example.org/bird-ontology#>
    SELECT ?bird WHERE {
        ?bird a bird:Species ;
              bird:hasSize "small" ; bird:hasColor "red" ; bird:inhabits "forests" ; bird:hasAnatomicalFeature "wings" .
    }
    
